# Augmentation by Albumentation

The core tenet of data centric machine learning is prioritizing high-quality, diverse, and relevant data for training models, recognizing its crucial role in model performance and generalization.

One technique that can help you with this is _augmentation_. Augmentation is the process of expanding a dataset by applying transformations like rotation, scaling, or noise to the data. This can help increase diversity in datasets, which in turn helps with generalization.

Augmentation is frequently associated with image data, but not exclusive to it. For instance, chemical applications of ML, augmentations such as conformer generation (different spatial arrangements of atoms in molecules) are used frequently. We will stick to images today.

At a first glance, augmentation looks very simple. It's really just a few basic transformations on an image. However, there are many pitfalls once masks, labels, boxes etc. get involved. This is why you're better off to use a dedicated augmentation library instead of building your own augmentations from scratch. One such library is [Albumentations](https://albumentations.ai).

In their own words,

> Albumentations is a fast and flexible image augmentation library. The library is widely used in industry, deep learning research, machine learning competitions, and open source projects. Albumentations is written in Python, and it is licensed under the MIT license. The source code is available at <https://github.com/albumentations-team/albumentations>.

You can install `albumentations` from `pip` like any regular python package. We've already taken care of this in the conda environment.

## Import the required libraries

To start our augmentation adventure, we of course need the `albumentations` library. On top of this, we also require a library to read images from the disk. Here, we'll use [`pillow`](https://pillow.readthedocs.io/).

In [ ]:
%pip install albumentations pillow
import albumentations as A
from PIL import Image
import numpy as np

## Augmentation pipelines

Albumentations is built around the concept of an "augmentation pipeline". If you have ever worked with [PyTorch transforms](https://pytorch.org/tutorials/beginner/basics/transforms_tutorial.html) or [`torchvision.transforms`](https://pytorch.org/vision/stable/transforms.html), the syntax should remind you of something.

To define an augmentation pipeline, you need to create an instance of the `Compose` class. `Compose` takes a list of transformations (or "augmentations") to apply.

In [ ]:
transform = A.Compose([
    A.RandomCrop(width=256, height=256),  # randomly crop the image to 256x256
    A.HorizontalFlip(p=0.5), # horizontally flip 50% of the images
    A.RandomBrightnessContrast(p=0.2), # randomly adjust brightness and contrast in 20% of the images
])

This pipeline performs three augmentations. It randomly crops the image to 256x256, flips 50% of the images horizontally, and randomly changes brightness and contrast in 20% of the images. You can find a list of all available augmentations [in the docs](https://albumentations.ai/docs/api_reference/augmentations/).

The parameters of the individual augmentations depend on the type of augmentation, but there is a recurring one: `p`. `p` is a special parameter that is supported by almost all augmentations. It controls the probability of applying the augmentation. `p=0.3` indicates a 30% probability of applying the augmentation.

Here is a visualized version of the augmentation pipeline, curtesy of the [albumentations documentation](https://albumentations.ai/docs/getting_started/image_augmentation/#step-2-define-an-augmentation-pipeline).

![image.png](../imgs/augmentation_pipeline_visualized.jpg)

## Read and transform images

Once you have created you augmentation pipeline, all that remains is reading and transforming your images.

In [ ]:
import matplotlib.pyplot as plt

# Read the image with PIL
image = np.array(Image.open("../imgs/cat.jpg"))

plt.imshow(image)
plt.axis('off')
plt.show()


Then, transforming it is just as easy!

In [ ]:
transformed = transform(image=image)
transformed_image = transformed["image"]
plt.imshow(transformed_image)

And that's it!

Here are a few more examples you can look at if you want to learn more about albumentations:

- [Defining a simple augmentation pipeline for image augmentation](https://albumentations.ai/docs/examples/example/)
- [Weather augmentations in Albumentations](https://albumentations.ai/docs/examples/example_weather_transforms/)
- [Cool augmentation examples on a diverse set of images from various real-world tasks](https://albumentations.ai/docs/examples/showcase/)

## Augmentation at scale

When working with large amounts of data, you'll want to persist the augmentations and not compute them on the fly, every time you use the dataset.
In the last section on Albumentations, we will combine augmentation with what we have learnt about data versioning in the previous part.

Below you find a simple python script to run Albumentations from the command line.

In [ ]:
%%writefile ../apply_augmentation.py
import os
import argparse
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import numpy as np


def apply_augmentation(image_path, save_dir, augmentation):
    image = Image.open(image_path)
    augmented = augmentation(image=np.array(image))
    augmented_image = Image.fromarray(augmented['image'])
    image_name = os.path.basename(image_path)
    augmented_image.save(os.path.join(save_dir, image_name))


def main(args):
    augmentation = getattr(A, args.augmentation)(**args.augmentation_params)

    if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)

    for filename in os.listdir(args.input_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(args.input_dir, filename)
            apply_augmentation(image_path, args.save_dir, augmentation)


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Apply Albumentations augmentations to a directory of images.")
    parser.add_argument("input_dir", type=str, help="Path to the directory containing input images.")
    parser.add_argument("save_dir", type=str, help="Path to the directory to save augmented images.")
    parser.add_argument("--augmentation", type=str, default="HorizontalFlip",
                        choices=[name for name in dir(A) if name[0].isupper()],
                        help="Name of the augmentation class.")
    parser.add_argument("--augmentation_params", type=str, nargs='*', default=[],
                        help="Parameters for the augmentation in the format key1=value1 key2=value2 ...")

    args = parser.parse_args()

    args.augmentation_params = dict(item.split('=') for item in args.augmentation_params)
    args.augmentation_params = {key: float(value) for key, value in args.augmentation_params.items()}

    main(args)


To run this script, you can use the command like this:

```shell
python script.py input_directory output_directory --augmentation AugmentationName --augmentation_params param1=value1 param2=value2 ...
```

It is a bit limited in as far as it can only apply a single augmentation in each run, but you can of course re-run it on the already augmented data and "emulate" `A.Compose` in this way.

---

## A little bit more on data version control

Use the script to augment the `102flowers` data! Here are a few example transformations to get you started.
Apply horizontal flip augmentation to images in the "input_images" directory and save the augmented images to the "augmented_images" directory:

In [ ]:
python apply_augmentations.py input_images augmented_images --augmentation HorizontalFlip

Apply random crop augmentation with width 200 and height 200 to images in the "input_images" directory and save the augmented images to the "augmented_images" directory:


In [ ]:
python apply_augmentations.py input_images augmented_images --augmentation RandomCrop --augmentation_params width=200 height=200

Apply random brightness and contrast augmentation to images in the "input_images" directory and save the augmented images to the "augmented_images" directory:

In [ ]:
python apply_augmentations.py input_images augmented_images --augmentation RandomBrightnessContrast

Most transformations are supported. Don't forget to set the `p` parameter, else you'll override all images. Augmenting all images could take a very long time, feel free to just abort the script at some point.

Once you are happy with your augmentations, you can commit the changes using DVC:

In [ ]:
dvc diff # check what has changed

Follow this up by the actual commit.

In [ ]:
dvc commit
git add data/102flowers.dvc
git commit -m "Add augmentations to the input images"

Note that the data version is tied to the `*.dvc` file!

When you want to checkout an old version of the data, you first checkout the Git commit that corresponds to this data version (in our case, we can use `HEAD^~1` as this refers to the second most recent commit) and only then checkout the data using `dvc checkout`.